# Assignment 5: Topic Models

Due: Tuesday, November 5.

This assignment has three problems. The first is about Bayesian inference. The second two are about topic models. You will first work with abstracts of scientific articles. These abstracts are obtained from arXiv.org, an open access repository for e-prints of articles in scientific fields maintained by Cornell University. You will then work with a collection of movie plots.

*For your convenience, we have separated the problems into three notebooks: assn5_problem1.ipynb, assn5_problem2.ipynb, and assn5_problem3.ipynb. Submit your solutions in these three notebooks, printing out each as a separate pdf.*

We provide significant "starter code" as discussed in lecture. We then ask you build topic models using the Python library gensim, and do some analysis over the topics obtained.

We ask that you please at least start the assignment right away. If you have any difficulties running gensim we would like to know!


# Problem 3: Topic Models on Movie Plots 

In this problem we will continue working with topic models, but this time with a new dataset. Instead of abstracts of scientific articles, we will create topic models over movie plot descriptions. This is a dataset containing descriptions of movies from Wikipedia. The dataset was [obtained](https://www.kaggle.com/jrobischon/wikipedia-movie-plots) from Kaggle, an online community of data scientists. We again provide extensive starter code to process the data.

Spoiler alert! We will use the movie "[Husbands and Wives](https://en.wikipedia.org/wiki/Husbands_and_Wives)" as a running example...

[<img width=200 src="https://upload.wikimedia.org/wikipedia/en/7/70/Husbands_moviep.jpg">](https://en.wikipedia.org/wiki/Husbands_and_Wives)


In [ ]:
import numpy as np
import re
import gensim
import pandas as pd
from collections import Counter

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
logging.root.level = logging.CRITICAL 

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# direct plots to appear within the cell, and set their style
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

This time around, the movie plot descriptions are in a CSV format in `movie_plots.csv`. The file is hosted on the Amazon Web Service s3. We'll use the `datascience` package to read this CSV file.



In [ ]:
filename = "https://s3.amazonaws.com/sds171/labs/lab07/movie_plots.csv"
data = pd.read_csv(filename)
data.head(5)

To make the data a little more manageable, we restrict to movies that were released after 1980. We then pull out the titles and plots as lists, for convenience.

In [ ]:
movies = data[data['Release Year'] > 1980]
titles = list(movies['Title'])
plots = list(movies['Plot'])

In [ ]:
movies.head()


In [ ]:
sample = 2015
print("Number of movies: %d\n" % movies.shape[0])
print("Plot of \"%s\":\n" % titles[2015])
print(plots[2015])


This plot description is from the movie "[Husbands and Wives](https://en.wikipedia.org/wiki/Husbands_and_Wives)"

We don't have LaTeX markup in these documents, but we'll still use some regular expressions to do some simpe pre-processing of punctuation. There are lots of names in the plot descriptions, so we'll remove all the words that have a capitalized first letter. This will remove lots of non-name words as well, but this'll be sufficient for our goal of building a basic topic model.

In [ ]:

# replace '-' with ' ', then remove punctuation
plots = [re.sub('-', ' ', plot) for plot in plots]
plots = [re.sub('[^\w\s]', '', plot) for plot in plots]

# remove tokens with a capitalized first letter 
# (broad stroke to remove names)
plots = [re.sub('[A-Z]\w*', '', plot) for plot in plots]
# replace multiple spaces by a single space
plots = [re.sub('[ ]+', ' ', plot) for plot in plots]

print(plots[sample])


Now, we further process each plot description by converting it to lower case, stripping leading and trailing white space, and then tokenizing by splitting on spaces.

In [ ]:
plots_tok = []
for plot in plots:
    processed = plot.lower().strip().split(' ')
    plots_tok.append(processed)

### 3.1 Further cleaning

As in problem 2, we will remove tokens that have digits, possessives or contractions, or are empty strings.
- `is_numeric(string)` checks if `string` has any numbers
- `has_poss_contr(string)` checks if `string` has possessives or contractions
- `empty_string(string)` checks if `string` is an empty string
- `remove_string(string)` checcks if `string` should be removed

In [ ]:
def is_numeric(string):
    return False

def has_poss_contr(string):
    return False

def empty_string(string):
    return False

def remove_string(string):
    return is_numeric(string) | has_poss_contr(string) | empty_string(string)

In [ ]:
temp = []
for plot in plots_tok:
    filtered = []
    for token in plot:
        if not remove_string(token):
            filtered.append(token)
    temp.append(filtered)
plots_tok = temp

Recall that to build topic models, we require the following components:
- A vocabulary of tokens that appear across all documents.
- A mapping of those tokens to a unique integer identifier, because topic model algorithms treat words by these identifiers, and not the strings themselves. For example, we represent `'epidemic'` as `word2id['epidemic'] = 50`
- The corpus, where each document in the corpus is a collection of tokens, where each token is represented by the identifier and the number of times it appears in the document. For example, in the first document above the token `'epidemic'`, which appears twice, is represented as `(50, 2)`

Now we will build a vocabulary representing the tokens that have appeared across all the plot descriptions we have. 


Recall that we can use the `Counter` class to build the vocabulary. The `Counter` is an extension of the Python dictionary, and also has key-value pairs. For the `Counter`, keys are the objects to be counted, while values are their counts.

In [ ]:
vocab = Counter()
for plot in plots_tok:
    vocab.update(plot)

print("Number of unique tokens: %d" % len(vocab))

Recall that removing rare words helps prevent our vocabulary from being too large. Many tokens appear only a few times across all the plot descriptions. Keeping them in the vocabulary increases subsequent computation time. Furthermore, their presence tends not to carry much significance for a document, since they can be considered as anomalies.

We remove rare words by only keeping tokens that appear more than 25 times across all plot descriptions.

In [ ]:
tokens = []
for token in vocab.elements():
    if vocab[token] >= 50:
        tokens.append(token)
vocab = Counter(tokens)

print("Number of unique tokens: %d" % len(vocab))

Recall that stop words are defined as very common words such as `'the'` and `'a'`. Removing stop words is important because their presence also does not carry much significance, since they appear in all kinds of texts.

We will remove stop words by removing the 200 most common tokens across all the plot descriptions.

In [ ]:
stop_words = []
for item in vocab.most_common(200):
    stop_word = item[0]
    stop_words.append(stop_word)
tokens = []
for token in vocab.elements():
    if token not in stop_words:
        tokens.append(token)
vocab = Counter(tokens)

print("Number of unique tokens: %d" % len(vocab))

Now we create a mapping for tokens to unique identifiers. 

In [ ]:
items = vocab.items()
id2word = {}
word2id = {}
idx = 0
for word, count in vocab.items():
    id2word[idx] = word
    word2id[word] = idx
    idx += 1
    
print("Number of tokens mapped: %d" % len(id2word))
print("Identifier for 'photograph': %d" % word2id['photograph'])
print("Word for identifier %d: %s" % (word2id['photograph'], id2word[word2id['photograph']]))

Now, we will remove, for each plot description, the tokens that are not found in our vocabulary.

In [ ]:
temp = []
for plot in plots_tok:
    filtered = []
    for token in plot:
        if token in vocab:
            filtered.append(token)
    temp.append(filtered)
plots_tok = temp

Let's create the corpus. Recall that the corpus should have the format
```
[(1841, 2), (2095, 2), (2096, 1), (2097, 1), (2098, 2), (105, 2), (2099, 1), (2100, 1), (270, 2), (1763, 1), (1870, 1), (2101, 1), (2017, 4), (633, 1), (1270, 1), (1093, 1), (2102, 1), (1197, 1), (113, 1), (1583, 1), (2103, 1), (2104, 2), (2105, 1), (873, 1), (1950, 1), (107, 1), (2106, 1), (2107, 1), (116, 1), (1436, 1), (62, 1), (2108, 1), (213, 1), (2109, 1), (1205, 1), (2110, 1), (1042, 1), (1275, 1), (1259, 1), (1342, 1), (2111, 1), (440, 1), (1662, 1), (374, 1), (663, 1)]
```

where each element is a pair containing the identifier for the token and the count of that token in just that plot description.

In [ ]:
corpus = []
for plot in plots_tok:
    plot_count = Counter(plot)
    corpus_doc = []
    for item in plot_count.items():
        pair = (word2id[item[0]], item[1])
        corpus_doc.append(pair)
    corpus.append(corpus_doc)

print("Plot, tokenized:\n", plots_tok[sample], "\n")
print("Plot, in corpus format:\n", corpus[sample])

Now, we are ready to create our topic model!

We again use gensim, a Python library to create topic models. Also, we again use the algorithm called latent dirichlet allocation implemented in the gensim library. 

**This step takes about 2 minutes**

In [ ]:
#%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

## Topics for Movies

Your task is now to carry out the same steps as for problem 2 (arXiv abstracts), but now for this dataset of movie plots

#### 3.2 Label the Topics
Label all the 10 topics with your interpretation of what the topics are. 

#### 3.3 Table of Topics for Movies
Create a function `create_movie_table(data, abstracts, corpus, lda_model)` which does the following:
- Goes through every movie plot and finds the most likely topic for that plot.
- Creates a table `movie_table` that has the following columns
    - `title`: the title of the movie
    - `topic`: the topic number of the most likely topic for each abstract
    - `label`: the topic label of that topic number, which you assigned in part 1
    - `prob`: the probability of that topic number
    - `plot`: a string containing the first 200 characters of the plot
- Show the first 10 rows of the table, then return the table

#### 3.4 Analysis for selected movies
Choose at least five movies, including 'Husbands and Wives' and discuss how the assignment of topics either does or does not make sense, according to your own understanding of the movies.
Note that Wikipedia pages are given for most of the movies in the original data. For example, 
[https://en.wikipedia.org/wiki/Absence_of_Malice](https://en.wikipedia.org/wiki/Absence_of_Malice) is the page for "Absence of Malice"

#### 3.5 Extra credit: Improve the model
For extra credit, improve the topic model by improving the processing of the data and the vocabulary, and selecting a more appropriate number of topics. Describe how your new model gives an improvement over the "quick and dirty" topic model built above.
